In [1]:
from datetime import datetime, timedelta
import pandas as pd
import sqlite3
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt
import ta
import yfinance as yf
import seaborn as sns
import numpy as np
from queue import Queue
import holidays
import scipy as sp
from itertools import combinations
from pymongo import MongoClient
import swifter

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from helper.client import *
from helper.machinelearner import *
from helper.dbmgr import *
from helper.dataprocessor import *

# Create an instance of CoinbaseClient
client = CoinbaseClient(api_key, api_secret)

In [ ]:
db_uri = 'mongodb://localhost:27017/'  
db_name = 'crypto_data'  

# Get and format data from CB

In [ ]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["customers"]
mydict = { "name": "John", "address": "Highway 37" }

x = mycol.insert_one(mydict)

print(x.inserted_id)

65e1ebe181d7df0b9cf67366


In [ ]:
# unique_pids = client.get_unique_product_ids()
# unique_pids = [f"{i}-{'USDC'}" for i in unique_pids]

# # start_time = datetime(2022, 10, 1)
# start_time = datetime(2024, 2, 21)
# end_time = datetime(2024, 2, 22)

# db_manager = DatabaseManager(start_time=start_time, end_time=end_time)
# print(db_manager.check_row_counts())

# db_manager.main(unique_pids)

# Technical Indicator creation

In [ ]:
fe = FeatureEngineer(db_name)
processed_data = fe.run()

Processing Collections:   0%|          | 0/6 [00:00<?, ?it/s]


KeyError: 'Columns not found: 0.7732289999999999, 1.087634, 1.42485, 1.8995700000000002, 4.859916, 5.9615800000000005, 1.5973650000000001, 7.325469999999999, 8.289235999999999, 9.805590000000002, 8.077532999999999, 11.096078, 8.985724, 6.076700000000001, 14.539140000000002, 2574.260073, 2321.1076959999996, 19.165575999999998, 20.383118, 20.959198999999998, 276.933066, 23.701990999999996, 1531.7191639999999, 0.743516, 26.578812000000003, 26.467078, 2077.6595279999997, 33.641856, 34.280824, 35.117775, 261.10022399999997, 37.753254, 290.11150000000004, 4136.305727999999, 40.49988199999999, 42.622965, 45.8479, 1073.890742, 53.31147000000001, 1.1201160000000001, 60.60702, 2365.3144209999996, 2110.815518, 3394.8165599999998, 1858.314991, 0.8790600000000001, 331.978185, 81.77957200000002, 0.8379839999999998, 1877.778676, 86.65269599999999, 90.218205, 91.276263, 93.940963, 350.73040799999995, 97.62862599999998, 1378.025649, 99.479124, 364.363023, 624.7393189999999, 114.815948, 892.61712, 4734.

# Preprocessing creation
- Labels
- Scaling
- Normalisation
- Stationary features

In [ ]:
data_preparator = DataPreparator(db_name)

# Prepare the data
final_data, non_stationary_columns = data_preparator.prepare_data()

In [ ]:
### label generation

label_generator = LabelGenerator(mongo_uri='mongodb://localhost:27017/',
                                  db_name='crypto_data', account_value=5000,
                                    tx_fee=0.006, profit_level=0.003)

label_generator = LabelGenerator(mongo_uri='mongodb://localhost:27017/', db_name='crypto_data', account_value=5000, tx_fee=0.006, profit_level=0.003)

# Specify the collection name where you want to save the labeled data
collection_name = 'labeled_crypto_data'

# Get the list of instruments from the database
instruments = label_generator.get_instruments()

# Iterate through each instrument and generate labels
for instrument in instruments:
    # Fetch data for the instrument
    df = label_generator.fetch_data(instrument)

    # Check if there's enough data to process
    if not df.empty:
        # Process the data to generate labels for this specific instrument
        for granularity in label_generator.granularities:
            df_granularity = df[df['granularity'] == granularity]

            for window_size in [15, 30, 45, 60]:
                if not df_granularity.empty:
                    result_df = label_generator.calculate_targets(df_granularity.copy(), window_size)
                    result_df['instrument'] = instrument
                    result_df['granularity'] = granularity
                    result_df['window_size'] = window_size

                    # Save the labeled data for this instrument to the MongoDB collection
                    label_generator.save_to_database(result_df, collection_name)

                    # Optional: Print a message to confirm data has been saved
                    print(f"Saved labeled data for {instrument} with granularity {granularity} and window size {window_size} to {collection_name}")


# Machine Learning on one set of parameters
- Grid Search for best parameters
- view feature importance on the best set and trim

# Save best model for backtesting

# Backtester